In [269]:
import re
import pandas as pd
import tqdm, time

#### 1. generate a txt file with format chrN:12345678-12345678 using chromosome and hg19_position from file all_variants_and_proxies.csv

In [3]:
df = pd.read_csv('~/Documents/Materials/iron/all_variants_and_proxies.csv')
# df['format'] = 'chr'+df.chromosome.astype(str)+':'+df.hg19_position.astype(str)+'-'+df.hg19_position.astype(str)
# df['format'].to_csv('~/Documents/Projects/iron/variants_hg19.txt', index=False) 

#### 2. convert hg19 coordinates to hg38 coordinates using liftover tool: 
https://genome.ucsc.edu/cgi-bin/hgLiftOver

#### 3. rename the file downloaded from the tool as variants_hg19.bed and then add it to all_variants_and_proxies.csv

In [7]:
df0 = pd.read_csv('~/Documents/Materials/iron/variants_hg38.bed', header=None) 
df['hg38_position'] = df0[0].str.extract('\:(\d+)\-') 
df.to_csv('~/Documents/Materials/iron/all_variants_and_proxies.csv', index=False)

#### 4. scrape off head and AF from https://results.finngen.fi/ using rsid
backend of finngen dashboard: https://github.com/FINNGEN/saige-pipelines/blob/master/wdl/gwas/saige.json

e.g. Given rs1961437,

```head = '22:37114993:A:G (rs1961437)'``` 

```str_af = 'AF 9.1e-1(ranges from 9.1e-1 to 9.1e-1 across all phenotypes)'```

```af = 0.91```

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
browser = webdriver.Chrome(Service('/Users/feiwang/Documents/Projects/chromedriver'))
browser.get('https://results.finngen.fi/')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [160]:
finngen_format_list, finngen_af_list = [], []

for i in tqdm.tqdm(df.rsid.tolist()):
#     browser.get('https://results.finngen.fi/variant/'+1:22227885-T-C)
    # there are 9 SNPs without rsid in the file
    if type(i) == float:
        finngen_format_list.append('id isna')
        finngen_eaf_list.append('id isna')
    else:
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(i)
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(Keys.RETURN)
        time.sleep(5)
        try:
            head = browser.find_element(By.XPATH, '/html/body/div/div[1]/div/h1').text
        except:
    #         err = browser.find_element_by_xpath('//*[@id="reactEntry"]/div/div/div/p[1]').text
            head = 'id notfound'
        if head == 'id notfound':
            eaf = 'id notfound'
        else:
            try:
                str_af = browser.find_element(By.XPATH, '//*[@id="maf-range"]/p').text
                af = float(re.findall('AF ([\d|\.|e|\-]+)\(ranges', str_eaf)[0])
            except:
                af = 'eaf notfound'

        finngen_format_list.append(head)
        finngen_af_list.append(af)
        time.sleep(1)

  0%|          | 0/1039 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
100%|██████████| 1039/1039 [1:50:57<00:00

In [169]:
df['finn_format'] = finngen_format_list
df['finn_af'] = finngen_af_list

In [198]:
df['finn_rsid'] = df.finn_format.str.extract(' \((rs\d+)\)')
df['finn_ref'] = df.finn_format.str.extract(':(\w+):\w+ \(')
df['finn_alt'] = df.finn_format.str.extract(':\w+:(\w+) \(')

In [199]:
df['match'] = df.finn_rsid == df.rsid

In [204]:
for i, row in df[(df.match == False)&(df.finn_rsid.isna())&(df.finn_format != 'id notfound')].iterrows():
    id_list = re.findall('(rs\d+)',row.finn_format)
    if row.rsid in id_list:
        df.loc[i,'finn_rsid'] = row.rsid
        df.loc[i,'match'] = True

In [245]:
print(len(df[df.match == False]),'cannot be found unfortunately')
print(len(df)-len(df[df.match == False]),'were found successfully')

514 cannot be found unfortunately
2672 were found successfully


#### 5. create a list of format that can be recognized in finngen sandbox

In [241]:
df['finn_hg38'] = df.finn_format.str.extract('\d{1,2}\:(\d+)\:\w+\:')
df['sandbox_format'] = 'chr'+df.chromosome.astype(str)+'_'+df.finn_hg38+'_'+df.finn_ref+'_'+df.finn_alt
df.to_csv('~/Documents/Materials/iron/all_variants_and_proxies.csv', index=False)

#### 6. obtain phecode_map from PheWAS package in R
See phecode_map.R for detail
#### 7. label if a PheCode is sex-specified
Download sex-specified PheCode dictionary from https://phewascatalog.org/files/phecode_definitions1.2.csv.zip

In [411]:
phecode_map = pd.read_csv('~/Documents/Materials/iron/phecode_map.csv') 
phecode_def = pd.read_csv('~/Documents/Materials/iron/phecode_definitions1.2.csv') 
phecode_list = phecode_map.phecode.unique().tolist()
phecode_dict = pd.DataFrame({'phecode':phecode_list})
phecode_dict = phecode_dict.merge(phecode_def[['phecode','sex']], 'left')

In [416]:
phecode_sex = phecode_dict[(~phecode_dict.sex.isna())&(phecode_dict.sex != 'Both')]
# in total 166 sex-specified phecodes
phecode_sex.to_csv('~/Documents/Materials/iron/phecode_sex.csv', index=None)

#### 8. upload the files to green bucket: 
https://console.cloud.google.com/storage/browser/fg-production-sandbox-6_green

gsutil cp ~/Documents/Materials/iron/all_variants_and_proxies.csv gs://fg-production-sandbox-6_green/FeiyiWang

gsutil cp ~/Documents/Materials/iron/phecode_map.csv gs://fg-production-sandbox-6_green/FeiyiWang

gsutil cp ~/Documents/Materials/iron/phecode_sex.csv gs://fg-production-sandbox-6_green/FeiyiWang

In [328]:
import statsmodels.api as sm
from sklearn import datasets

In [332]:
X = sm.add_constant(x_train)
est = sm.Logit(y_train, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Wed, 09 Mar 2022   Pseudo R-squ.:                 0.08292
Time:                        11:10:29   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
Covariance Type:            nonrobust   LLR p-value:                 7.578e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9900      1.140     -3.500      0.000      -6.224      -1.756
gre            0.0023      0.

In [387]:
a = est.summary2().tables[1].loc['gre',:]
a

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
const,-3.989979,1.139951,-3.500132,0.000465,-6.224242,-1.755716
gre,0.002264,0.001094,2.069863,0.038465,0.000120,0.004409
gpa,0.804038,0.331819,2.423119,0.015388,0.153684,1.454391
2,-0.675443,0.316490,-2.134171,0.032829,-1.295751,-0.055135
3,-1.340204,0.345306,-3.881202,0.000104,-2.016992,-0.663416
4,-1.551464,0.417832,-3.713131,0.000205,-2.370399,-0.732529


In [405]:
b = pd.DataFrame(columns=['Coef.','Std.Err.','z','P>|z|','phecode'])
a = est.summary2().tables[1].loc['gre',['Coef.','Std.Err.','z','P>|z|']]
a['phecode'] = 8.1
b.append(a)

,Coef.,Std.Err.,z,P>|z|,phecode
gre,0.002264,0.001094,2.069863,0.038465,8.1


What tripped me up:

- disable sklearn regularization LogisticRegression(C=1e9)
- add statsmodels intercept sm.Logit(y,sm.add_constant(X)) OR disable sklearn intercept LogisticRegression(C=1e9,fit_intercept=False)
- sklearn returns probability for each class so model_sklearn.predict_proba(X)[:,1] == model_statsmodel.predict(X)
- Use of predict fucntion model_sklearn.predict(X) == (model_statsmodel.predict(X)>0.5).astype(int)

I'm now seeing the same results in both libraries.